# [실습] LangChain Tool Call과 Agent

LangChain의 기본 코드를 통해, Tool Call과 Agent가 작동하는 과정에 대해 알아보겠습니다.   

기본 라이브러리를 설치합니다.

In [17]:
!pip install langchain langchain_google_genai langchain_core langchain_community langchain_tavily

Google API Key를 설정합니다.

구글 로그인 후, https://aistudio.google.com/apikey 에서 발급받을 수 있습니다.

In [ ]:
import os

os.environ['GOOGLE_API_KEY'] = ''

Gemini 모델을 설정합니다.   
무료 API의 경우 분당 제한이 존재하므로, 랭체인의 Rate Limiter를 적용할 수 있습니다.

In [18]:
from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_openai import ChatOpenAI

# Gemini API는 분당 10개 요청으로 제한
# 즉, 초당 약 0.167개 요청 (10/60)
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.167,  # 분당 10개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=10,  # 최대 버스트 크기
)

# rate limiter를 LLM에 적용
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    rate_limiter=rate_limiter,
    # temperature
    # max_tokens

    thinking_budget = 500  # 추론(Reasoning) 토큰 길이 제한
)

In [19]:
llm.invoke("안녕? 너는 이름이 뭐니? 한 문장으로 말해줘.")

AIMessage(content='안녕하세요! 저는 이름이 없으며, Google이 훈련한 대규모 언어 모델입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--1d30f245-7a3e-43ca-a7c5-2c57a46159bd-0', usage_metadata={'input_tokens': 19, 'output_tokens': 462, 'total_tokens': 481, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 442}})

`invoke()`를 통해 llm에 입력을 전달합니다.    
랭체인 기본 클래스인 `Message` 계열 클래스를 직접 만들어 리스트로 전달하거나,   
프롬프트 템플릿을 통해 입력의 구성을 만들어 전달할 수 있습니다.

In [20]:
# 1. HumanMessage를 이용한 방법

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

system_msg = SystemMessage(content="""사용자의 아래 [질문]에 대해 친절하게 답변하세요.
답변의 길이는 5문장을 넘지 않도록 하고, 개조식으로 작성하세요.""")
msg = HumanMessage(content="안녕! 너는 이름이 뭐니?")

messages = [system_msg, msg]

response = llm.invoke(messages)
response

AIMessage(content='안녕! 저에 대해 궁금해해 주셔서 감사합니다.\n\n*   저는 개인적인 이름이 없어요.\n*   저는 Google에서 훈련한 대규모 언어 모델이랍니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--bb8962f5-1cae-4908-9935-2f4106881b98-0', usage_metadata={'input_tokens': 49, 'output_tokens': 468, 'total_tokens': 517, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 427}})

Chat 모델의 출력은 AIMessage라는 형식으로 변환됩니다.

In [21]:
print(response.content)

안녕! 저에 대해 궁금해해 주셔서 감사합니다.

*   저는 개인적인 이름이 없어요.
*   저는 Google에서 훈련한 대규모 언어 모델이랍니다.


In [ ]:
# 2. ChatPromptTemplate을 이용한 방법
from langchain_core.prompts import ChatPromptTemplate
# langchain.prompts --> langchain_core.prompts


# 입력 변수가 {question}인 템플릿
prompt = ChatPromptTemplate([
    ('system', '''사용자의 아래 [질문]에 대해 친절하게 답변하세요.
     답변의 길이는 5문장을 넘지 않도록 하고, 개조식으로 작성하세요.'''),
     ('user', '''[질문]: {question}''')])
prompt


ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='사용자의 아래 [질문]에 대해 친절하게 답변하세요.\n     답변의 길이는 5문장을 넘지 않도록 하고, 개조식으로 작성하세요.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='[질문]: {question}'), additional_kwargs={})])

프롬프트와 llm을 |로 연결하여 체인을 구성합니다.   
(랭체인 기본 문법으로, LangChain 기초 복습 자료에서 자세히 보실 수 있습니다!)

In [23]:
chain = prompt | llm
chain

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='사용자의 아래 [질문]에 대해 친절하게 답변하세요.\n     답변의 길이는 5문장을 넘지 않도록 하고, 개조식으로 작성하세요.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='[질문]: {question}'), additional_kwargs={})])
| ChatGoogleGenerativeAI(rate_limiter=<langchain_core.rate_limiters.InMemoryRateLimiter object at 0x000001D9191EE9F0>, model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), thinking_budget=500, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001D91C66F770>, default_metadata=(), model_kwargs={})

In [24]:
response = chain.invoke({'question':"랭체인과 랭그래프의 차이가 뭐야?"})

print(response.content)

랭체인과 랭그래프는 LLM 애플리케이션 개발을 위한 도구이지만, 다음과 같은 차이가 있습니다.

*   **랭체인 (LangChain):**
    *   LLM 애플리케이션 개발을 위한 포괄적인 프레임워크입니다.
    *   프롬프트 관리, 모델 연동, 체인 구성, 에이전트 생성 등 다양한 기능을 제공합니다.
    *   주로 순차적인(linear) 워크플로우를 구성하는 데 강점이 있습니다.

*   **랭그래프 (LangGraph):**
    *   랭체인 위에 구축된 라이브러리로, 특히 에이전트 워크플로우를 위한 것입니다.
    *   상태 관리와 반복적인(cyclical) 흐름을 그래프 형태로 정의하고 실행하는 데 특화되어 있습니다.
    *   복잡한 에이전트가 스스로 판단하고 여러 단계를 거쳐 문제를 해결하는 다중 턴 상호작용 구현에 유용합니다.


In [25]:
# 입력 변수가 1개일 때는 값만 invoke해도 됨

response = chain.invoke("패스트캠퍼스 랭그래프 강의 이름이 뭐야?")

print(response.content)

안녕하세요! 패스트캠퍼스 랭그래프 강의 이름은 다음과 같습니다.

*   **GPTs 개발을 위한 LangGraph & Agent**

이 강의는 LangGraph를 활용하여 GPTs 및 에이전트를 개발하는 방법에 초점을 맞추고 있습니다.


존재하지 않는 내용에 대한 환각이 발생한 모습입니다.

## LLM Tool 설정하기

Tool은 LLM이 사용할 수 있는 다양한 외부 기능을 의미합니다.   
LangChain에서 자체적으로 지원하는 Tool을 사용하거나, RAG의 Retriever를 Tool로 변환하는 것도 가능합니다.     
또한, 함수를 Tool로 변환할 수도 있습니다.

1. Tavily Search (http://app.tavily.com/)

Tavily는 AI 기반의 검색 엔진입니다. 계정별 월 1000개의 무료 사용량을 지원합니다.      
Tavily Search는 URL과 함께 내용의 간단한 요약을 지원하는 것이 특징입니다.


In [26]:
os.environ['TAVILY_API_KEY'] = 'tvly-xxxx'


In [32]:
from langchain_tavily import TavilySearch
# TavilySearchResults --> langchain_tavily의 TavilySearch로 변경

tavily_search = TavilySearch(
    max_results=3,
    include_answer = True,
    include_raw_content = 'markdown'
    # 더 많은 옵션은 Tavily API 문서 Playground 참고
    )
tavily_search

TavilySearch(max_results=3, include_answer=True, include_raw_content='markdown', api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'), api_base_url=None))

In [33]:
search_docs = tavily_search.invoke("랭그래프와 랭체인의 차이")
search_docs

{'query': '랭그래프와 랭체인의 차이',
 'follow_up_questions': None,
 'answer': 'LangGraph provides detailed state management, while LangChain offers more automated, implicit state management. LangGraph is better for complex workflows, whereas LangChain is simpler.',
 'images': [],
 'results': [{'url': 'https://wikidocs.net/261585',
   'title': '1-1-2. LangGraph와 LangChain의 차이점 - 위키독스',
   'content': 'LangGraph는 명시적이고 세밀한 상태 관리를 제공합니다. · LangChain은 상대적으로 암시적이고 자동화된 상태 관리를 제공합니다.',
   'score': 0.8884594,
   'raw_content': None},
  {'url': 'https://www.youtube.com/watch?v=yD3SnuU6Pd8',
   'title': 'EP 2. Langchain과 Langgraph의 차이점 - YouTube',
   'content': 'EP 2. Langchain과 Langgraph의 차이점\n모두의AI\n19100 subscribers\n206 likes\n9182 views\n7 Nov 2024\n안녕하세요, 오늘은 Langchain과 Langgraph의 차이점에 대해서 알아봅니다.\n많은 LLM 기반 어플리케이션의 경우 Langchain만으로 충분하지만, 더 복잡한 워크플로우와 에이전트 시스템을 구현하기 위해서는 Langgraph를 활용하는 것이 더욱 효율적입니다. 특히 최근 Agentic RAG가 점차 떠오르는데 이때 Langgraph가 중요한 역할을 합니다.\n5 comments\n',
   'score': 0.7613083,
   'raw

# LLM에 Tool 연결하기   

생성한 툴은 llm.bind_tools()를 통해 LLM에 연결할 수 있습니다.    
이 기능은 Gemini, GPT, Claude 등의 툴에서 지원하는데요.     
HuggingFace 공개 모델에서 Tool을 사용하는 경우는 이후 실습에서 자세히 알아보겠습니다!

In [34]:
tools = [tavily_search]

llm_with_tools = llm.bind_tools(tools)

llm_with_tools

RunnableBinding(bound=ChatGoogleGenerativeAI(rate_limiter=<langchain_core.rate_limiters.InMemoryRateLimiter object at 0x000001D9191EE9F0>, model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), thinking_budget=500, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001D91C66F770>, default_metadata=(), model_kwargs={}), kwargs={'tools': [{'type': 'function', 'function': {'name': 'tavily_search', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. It not only retrieves URLs and snippets, but offers advanced search depths, domain management, time range filters, and image search, this tool delivers real-time, accurate, and citation-backed results.Input should be a search query.', 'parameters': {'properties': {'query': {'description': 'Search query to look up', 'type': 'string'}, 'include_domains': {'a

랭체인에서, tool 정보는 tools에 저장되는데요.   
해당 내용은 랭체인 내부에서 json Schema 형식으로 프롬프트에 포함됩니다.

In [36]:
# 시스템 메시지에 포함되는 내용
# (`LLM과 Agent` 강의 슬라이드를 참고하세요!)

llm_with_tools.kwargs['tools']

[{'type': 'function',
  'function': {'name': 'tavily_search',
   'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. It not only retrieves URLs and snippets, but offers advanced search depths, domain management, time range filters, and image search, this tool delivers real-time, accurate, and citation-backed results.Input should be a search query.',
   'parameters': {'properties': {'query': {'description': 'Search query to look up',
      'type': 'string'},
     'include_domains': {'anyOf': [{'items': {'type': 'string'},
        'type': 'array'},
       {'type': 'null'}],
      'default': [],
      'description': 'A list of domains to restrict search results to.\n\n        Use this parameter when:\n        1. The user explicitly requests information from specific websites (e.g., "Find climate data from nasa.gov")\n        2. The user mentions an organization or company without spe

LLM은 프롬프트로 주어지는 툴 정보를 바탕으로 Tool의 사용을 결정합니다.    
Schema를 통해, 툴의 의미와 사용 방법, 형식을 이해합니다.

이후, LLM은 Tool 사용이 필요한 경우 특별한 포맷의 메시지를 출력합니다.   
이를 Tool Call Message라고 부릅니다.

In [37]:
tool_chain = prompt | llm_with_tools
tool_call_msg = tool_chain.invoke("패스트캠퍼스의 '랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'은 어떤 과정이야?")
tool_call_msg


AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search', 'arguments': '{"query": "\\ud328\\uc2a4\\ud2b8\\ucea0\\ud37c\\uc2a4 \'\\ub7ad\\uadf8\\ub798\\ud504\\ub85c \\ud55c\\ubc88\\uc5d0 \\uc644\\uc131\\ud558\\ub294 \\ubcf5\\uc7a1\\ud55c RAG\\uc640 Agent \\uacfc\\uc815\'"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--0e0703f4-10f0-49c7-a740-8861bdd77f7c-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': "패스트캠퍼스 '랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'"}, 'id': '5c248c6d-eee7-474d-8ac0-22791fc36553', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1607, 'output_tokens': 90, 'total_tokens': 1697, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 52}})

In [38]:
tool_call_msg = llm_with_tools.invoke("패스트캠퍼스의 '랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'은 어떤 과정인지 검색해서 알려줘")
tool_call_msg

AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search', 'arguments': '{"query": "\\ud328\\uc2a4\\ud2b8\\ucea0\\ud37c\\uc2a4 \\ub7ad\\uadf8\\ub798\\ud504\\ub85c \\ud55c\\ubc88\\uc5d0 \\uc644\\uc131\\ud558\\ub294 \\ubcf5\\uc7a1\\ud55c RAG\\uc640 Agent \\uacfc\\uc815"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--a27a2867-0766-41db-a181-0bd0f1bd5c68-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': '패스트캠퍼스 랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'}, 'id': '7fa56ad9-f73a-47e6-be05-c8b1eb767308', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1567, 'output_tokens': 105, 'total_tokens': 1672, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 68}})

In [39]:
tool_call_msg.tool_calls
# 리스트 형식인 이유는? 여러 개의 툴을 동시에 사용할 수 있기 때문

[{'name': 'tavily_search',
  'args': {'query': '패스트캠퍼스 랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'},
  'id': '7fa56ad9-f73a-47e6-be05-c8b1eb767308',
  'type': 'tool_call'}]

`tool_calls`에 포함된 name을 활용하면 툴 결과를 전달할 수 있습니다.   
`name` 값은 문자열이므로, dictionary를 통해 연결합니다.

In [41]:
tool_list ={'tavily_search': tavily_search}

tool_name = tool_call_msg.tool_calls[0]['name']
tool_exec = tool_list[tool_name]

tool_name, tool_exec

('tavily_search',
 TavilySearch(max_results=3, include_answer=True, include_raw_content='markdown', api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'), api_base_url=None)))

Tool에 tool_call을 입력해 invoke를 수행합니다.

In [42]:
tool_msg = tool_exec.invoke(tool_call_msg.tool_calls[0])
tool_msg

ToolMessage(content='{"query": "패스트캠퍼스 랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정", "follow_up_questions": null, "answer": "The course teaches complex RAG and Agent development using LangGraph. It lasts 18 hours and offers flexible online learning. Refunds are available within the first 7 days of enrollment.", "images": [], "results": [{"url": "https://cdn.day1company.io/prod/uploads/202509/100554-1064/-%ED%8C%A8%EC%8A%A4%ED%8A%B8%EC%BA%A0%ED%8D%BC%EC%8A%A4--%EA%B5%90%EC%9C%A1%EA%B3%BC%EC%A0%95%EC%86%8C%EA%B0%9C%EC%84%9C-%EB%9E%AD%EA%B7%B8%EB%9E%98%ED%94%84%EB%A1%9C-%ED%95%9C%EB%B2%88%EC%97%90-%EC%99%84%EC%84%B1%ED%95%98%EB%8A%94-%EB%B3%B5%EC%9E%A1%ED%95%9C-rag%EC%99%80-agent.pdf", "title": "[PDF] 랭그래프로 한번에 완성하는 복잡한 RAG와 Agent", "content": "Life Changing Education 랭그래프로 한번에 완성하는 복잡한 RAG와 Agent Life Changing Education Copyright ⓒ FAST CAMPUS Corp. 강의정보 강의장 온라인 강의 | 데스크탑, 노트북, 모바일 등 수강 기간 평생 소장 상세페이지 https://fastcampus.co.kr/data_online_langgraph 강의시간 약 18시간 문의 고객센터 강의특징 나만의 속도로 낮이나 새벽이나 내가 원하는 시간

이번에는 ToolMessage라는 새로운 형태의 메시지가 생성되었습니다!

툴 실행 결과를 LLM에게 다시 전달하여, 이를 해석하고 답변하게 만들어 보겠습니다.   

Chain 형식의 구성은 메시지를 변화시키는 것이 복잡하므로, 이전에 사용했던 `HumanMessage`에서 시작하여   
LLM의 답변인 `AIMessage`와 `ToolMessage` 를 추가합니다.

In [43]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="패스트캠퍼스의 '랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'은 어떤 과정이야?")]

messages.append(tool_call_msg)
messages.append(tool_msg)

messages

[HumanMessage(content="패스트캠퍼스의 '랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'은 어떤 과정이야?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search', 'arguments': '{"query": "\\ud328\\uc2a4\\ud2b8\\ucea0\\ud37c\\uc2a4 \\ub7ad\\uadf8\\ub798\\ud504\\ub85c \\ud55c\\ubc88\\uc5d0 \\uc644\\uc131\\ud558\\ub294 \\ubcf5\\uc7a1\\ud55c RAG\\uc640 Agent \\uacfc\\uc815"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--a27a2867-0766-41db-a181-0bd0f1bd5c68-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': '패스트캠퍼스 랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'}, 'id': '7fa56ad9-f73a-47e6-be05-c8b1eb767308', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1567, 'output_tokens': 105, 'total_tokens': 1672, 'input_token_details': {'cache_read': 0}, 'output_

`Query-Tool Call-Tool`의 형식은 가장 기본적인 툴 사용 방법입니다.

In [45]:
result = llm_with_tools.invoke(messages)
# 검색 결과를 바탕으로 답변한 모습

print(result.content)


[{'type': 'text', 'text': "패스트캠퍼스의 '랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'은 LangGraph를 활용하여 복잡한 RAG(검색 증강 생성) 및 AI 에이전트를 개발하는 방법을 가르치는 온라인 강의입니다.\n\n주요 특징은 다음과 같습니다:\n*   **학습 내용**: LangGraph의 원리, 활용법, Node, Edge, Graph 구조, 다양한 State 제어 방법 등을 다룹니다. 또한, LangGraph가 사용되는 6가지 대표적인 활용 사례를 통해 복잡한 RAG 시스템과 고도화된 AI 에이전트 플로우를 구축하는 방법을 배울 수 있습니다.\n*   **강의 시간**: 총 약 18시간 분량으로 구성되어 있습니다.\n*   **수강 방식**: 온라인 강의로, 데스크탑, 노트북, 모바일 등 다양한 기기에서 원하는 시간에 무제한으로 복습하며 수강할 수 있습니다.\n*   **대상**: LLM(대규모 언어 모델) 애플리케이션 개발에 필수적인 LangGraph를 배우고 싶거나, 복잡한 RAG와 AI 에이전트 개발에 관심 있는 분들을 위한 과정입니다.\n*   **강사**: 노토랩 대표이자 LLM 강의/개발/기술 자문을 맡고 있는 변형호 강사입니다.\n*   **환불 규정**: 수강 시작 후 7일 이내, 5강 미만 수강 시 100% 환불이 가능합니다.\n\n이 과정은 LangChain의 단일 기능을 넘어 복잡한 워크플로우를 구성하고 효과적으로 관리하는 데 필요한 LangGraph를 심층적으로 다루며, 실제 산업 분야에서 Advanced RAG를 구축하는 데 필요한 지식과 실습을 제공합니다.", 'extras': {'signature': 'CtIIAdHtim8OxkgsibVw4K8A83qE8fQyGDeKg2hZeRxiytez0+zG8/sFOofg2UyXGwIl+Ll6zYW4NW/JaOX+QdbP6Reid/UuoHdS1XptPE5IJSTv3Pm4J5kdj9/uD6F4llqB/srHdSkAT4eIf9VqJ0XLbWiaR+Hw+

이제, 지금까지 배운 내용을 종합하여 실행해 보겠습니다.   
질문이 주어지면, 질문에 대한 검색을 수행하여 결과를 바탕으로 답변합니다.   

<br><br>

그런데, 이번에는 분기가 필요합니다.   
검색이 필요없는 질문이라면, LLM은 tool을 요청하지 않을 것입니다.

In [46]:
# LLM, Question, Tool을 이용한 간단한 함수
def simple_workflow(llm, question , tools = [tavily_search]):

    # 툴과 LLM 구성

    tool_list = {x.name: x for x in tools}
    # tavily_search.name = 'tavily_search_results_json' 을 이용하면
    # tool_list = {'tavily_search_results_json': tavily_search} 와 동일합니다.

    llm_with_tools = llm.bind_tools(tools)


    # 메시지 구성
    messages = [HumanMessage(content=question)]
    print('Query:', question)


    # LLM에 메시지 전달 (분기)
    tool_msg = llm_with_tools.invoke(question)
    messages.append(tool_msg)

    if tool_msg.tool_calls:
        # 툴 호출이 있을 경우: 툴 실행 후 결과를 전달
        tool_name = tool_msg.tool_calls[0]['name']

        print(f"-- {tool_name} 사용 중 --")
        print(tool_msg.tool_calls[0])


        tool_exec = tool_list[tool_name]

        tool_result = tool_exec.invoke(tool_msg.tool_calls[0])
        messages.append(tool_result)
        result = llm_with_tools.invoke(messages)

    else:
        # 툴 호출이 없을 경우: 처음 출력을 그대로 전달
        result = tool_msg

    return result.content

In [48]:
response = simple_workflow(llm, "2025년 10월 출시된 OpenAI의 AI 브라우저 이름이 뭐야?")
print(response)

Query: 2025년 10월 출시된 OpenAI의 AI 브라우저 이름이 뭐야?
-- tavily_search 사용 중 --
{'name': 'tavily_search', 'args': {'end_date': '2025-10-31', 'topic': 'general', 'query': 'OpenAI AI browser name', 'start_date': '2025-10-01'}, 'id': '2f93a7b3-e23e-4d14-ab89-dce934241e7e', 'type': 'tool_call'}
2025년 10월에 출시된 OpenAI의 AI 브라우저 이름은 **Atlas**입니다. 이 브라우저는 OpenAI의 챗봇인 ChatGPT와 긴밀하게 작동하도록 설계되었습니다.


# Custom Tool 만들기

Tavily Search의 경우, 랭체인에서 사전에 구성한 기본 Schema가 존재하지만,   

Tool을 새로 만드는 경우에는 description과 같은 값을 직접 구성해야 합니다.   
가장 간단한 방식은 랭체인의 Tool 데코레이터를 사용하는 것입니다.  

In [49]:
from langchain_core.tools import tool

@tool
def multiply(x:int, y:int) -> int:
    "x와 y를 입력받아, x와 y를 곱한 결과를 반환합니다."
    return x*y

@tool
def current_date() -> str:
    "현재 날짜를 %y-%m-%d 형식으로 반환합니다."
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d")

print(multiply.invoke({'x':3, 'y':4}))
print(current_date.invoke({}))


12
2025-10-26


이전에 생성한 tavily_search 툴도 보다 깔끔하게 구성할 수 있습니다.

In [74]:
@tool
def tavily_search(query:str, max_results:int=5):
    """Tavily API를 통해 검색 결과를 가져옵니다.
query: 검색어
max_results : 검색 결과의 수(최소 1, 최대 20, 별도의 요청이 없으면 5로 고정)"""
    tavily_search = TavilySearch(max_results=max_results,     
                                include_answer = True,
                                include_raw_content = 'markdown'
    )

    search_results = tavily_search.invoke(query)

    context = f'POSSIBLE ANSWER: {search_results.get('answer')}\n\n' 
    for doc in search_results['results']:
        if doc.get('raw_content'):
            doc_content = doc.get('raw_content')
        else:
            doc_content = doc.get('content')
        context += 'TITLE: ' + doc.get('title','N/A') + '\nURL:' + doc.get('url')+ '\nContent:'+ doc_content + '\n---\n'
    return context

In [75]:
tools = [tavily_search, multiply, current_date]

In [76]:
llm.bind_tools(tools)

RunnableBinding(bound=ChatGoogleGenerativeAI(rate_limiter=<langchain_core.rate_limiters.InMemoryRateLimiter object at 0x000001D9191EE9F0>, model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), thinking_budget=500, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001D91C66F770>, default_metadata=(), model_kwargs={}), kwargs={'tools': [{'type': 'function', 'function': {'name': 'tavily_search', 'description': 'Tavily API를 통해 검색 결과를 가져옵니다.\nquery: 검색어\nmax_results : 검색 결과의 수(최소 1, 최대 20, 별도의 요청이 없으면 5로 고정)', 'parameters': {'properties': {'query': {'type': 'string'}, 'max_results': {'default': 5, 'type': 'integer'}}, 'required': ['query'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'multiply', 'description': 'x와 y를 입력받아, x와 y를 곱한 결과를 반환합니다.', 'parameters': {'properties': {'x': {'type': 'integer'}, 'y': {'type': 'integer'}}, 'required': ['x', 'y'], 'type': 'object'}}}, {'type': 'functi

In [77]:
question = '오늘 날짜는?'

result = simple_workflow(llm, question, tools)
result


Query: 오늘 날짜는?
-- current_date 사용 중 --
{'name': 'current_date', 'args': {}, 'id': 'aeb4542c-b7a1-45a3-ab30-6b53772a01f2', 'type': 'tool_call'}


'2025년 10월 26일입니다.'

In [78]:
# LLM은 오늘 날짜를 모른다
llm.invoke('오늘 날짜는?').content

'오늘 날짜는 2024년 5월 15일입니다.'

<br><br>
## [연습문제] 복권 숫자 예측시키기

아래의 함수에 대해, 함수의 설명을 추가하여 llm의 툴로 전달해 봅시다.

In [79]:
from typing import List
import random

@tool
def generate_random_numbers(min_val: int, max_val: int, count: int) -> List[int]:
    """주어진 범위 내에서 지정된 개수만큼의 중복되지 않은 랜덤 정수를 생성합니다.
    Args:
        min_val (int): 최솟값
        max_val (int): 최댓값
        count (int): 생성할 숫자의 개수

    Returns:
        List[int]: 중복되지 않은 랜덤 정수들의 리스트
    """
    # possible_range = max_val - min_val + 1
    # if count > possible_range:
    #     raise ValueError(f"생성 가능한 숫자의 범위({possible_range}개)보다 더 많은 숫자({count}개)를 요청했습니다.")

    return str(random.sample(range(min_val, max_val + 1), count))

In [80]:
# LLM에 함수 Bind

llm_lotto = llm.bind_tools([generate_random_numbers])
llm_lotto.invoke('로또 번호 추첨해줘! 1부터 45까지 6개를 뽑으면 돼.')

AIMessage(content='', additional_kwargs={'function_call': {'name': 'generate_random_numbers', 'arguments': '{"max_val": 45, "min_val": 1, "count": 6}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--c11ba3f9-93b2-471d-bd32-8f4fc482cb47-0', tool_calls=[{'name': 'generate_random_numbers', 'args': {'max_val': 45, 'min_val': 1, 'count': 6}, 'id': 'c7ee7d8f-8e86-4df7-868c-d5da0ec28966', 'type': 'tool_call'}], usage_metadata={'input_tokens': 167, 'output_tokens': 111, 'total_tokens': 278, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 79}})

In [81]:
# simple_workflow 함수를 이용하여 실행하기

simple_workflow(llm_lotto, '로또 번호 추첨해줘! 1부터 45까지 6개를 뽑으면 돼.', [generate_random_numbers])

Query: 로또 번호 추첨해줘! 1부터 45까지 6개를 뽑으면 돼.
-- generate_random_numbers 사용 중 --
{'name': 'generate_random_numbers', 'args': {'max_val': 45, 'min_val': 1, 'count': 6}, 'id': '7034f75b-2ac2-4008-b426-130ff7de42f5', 'type': 'tool_call'}


'생성된 로또 번호는 다음과 같습니다: 24, 37, 11, 38, 2, 26. 행운을 빕니다!'

우리가 만든 `simple_workflow` 함수는 가장 간단한 형태의 Single Tool Calling을 수행하는 방식이었는데요.   

실제 환경의 시나리오는 훨씬 복잡합니다.   
한 번에 여러 개의 툴을 실행하거나, 툴 실행 결과를 받아 다음 툴에 활용할 수도 있을 것입니다.   

랭체인에서도 이와 같이 복잡한 작업을 수행하기 위한 기능이 자체적으로 구현되어 있는데요.    

예시로 `Structured Chat Agent`를 간단하게 만들고 실행해 보겠습니다.

# Agent

생각-행동-관찰을 거치는 ReAct 에이전트는 복잡한 플로우를 효과적으로 처리합니다.   
- 생각(Thought): 주어진 Context 상에서 다음 작업을 어떻게 수행할지 설명하는 과정
- 행동(Action): Tool 실행 명령어를 생성하는 과정
- 관찰(Thought): Tool 결과를 Context에 추가하는 과정


이 과정은 랭체인에 구성된 `create_structured_chat_agent`를 이용해 구성할 수 있습니다.

In [85]:
# langchain_classic
# 2025년 10월 20일 출시된 랭체인 v 1.0 이후로는 기존 랭체인 기능이 langchain_classic으로 옮겨졌습니다.
# 만약 이전 버전을 쓰시는 경우, 아래의 langchain_classic을 langchain으로 바꿔주세요.
from langchain_classic.agents import AgentExecutor, create_structured_chat_agent
from langchain_core.prompts import ChatPromptTemplate

agent_prompt = ChatPromptTemplate([
    ('system','''
최대한 정확히 질문에 답변하세요. 당신은 다음의 툴을 사용할 수 있습니다:
{tools}

action 키 (tool name)와 action_input 키를 포함하는 json 형태로 출력하세요.

action의 값은 "Final Answer" 또는 {tool_names} 중 하나여야 합니다.
반드시 하나의 json 형태만 출력하세요. 다음은 예시입니다.
```
{{

  "action": $TOOL_NAME,

  "action_input": $INPUT

}}
```

아래의 포맷으로 답변하세요.:

Question: 최종적으로 답변해야 하는 질문
Thought: 무엇을 해야 하는지를 항상 떠올리세요.
Action:
```
$JSON_BLOB
```
Observation: 액션의 실행 결과
... (이 Thought/Action/Observation 은 10번 이내로 반복될 수 있습니다.)

Thought: 이제 답을 알겠다!
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
```
'''),
('user','''Question: {input}
Thought: {agent_scratchpad}''')])


agent = create_structured_chat_agent(llm, tools, agent_prompt)

In [86]:
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

agent_executor.invoke({'input':"Open Source Multimodal LLM Model 추천해줘"})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "tavily_search",
  "action_input": {
    "query": "Open Source Multimodal LLM Model recommendations"
  }
}
```POSSIBLE ANSWER: InternVL 3.5 is a top open-source multimodal model. Pixtral and GLM-4.5V are also leading open-source options. Qwen 2.5 VL 72B Instruct is notable for vision-language integration.

TITLE: InternVL 3.5 : Best open-sourced Multi-Modal LLM - Medium
URL:https://medium.com/data-science-in-your-pocket/internvl-3-5-best-open-sourced-multi-modal-llm-bc929e2b6338
Content:[Sitemap](/sitemap/sitemap.xml)

[Open in app](https://rsci.app.link/?%24canonical_url=https%3A%2F%2Fmedium.com%2Fp%2Fbc929e2b6338&%7Efeature=LoOpenInAppButton&%7Echannel=ShowPostUnderCollection&%7Estage=mobileNavBar&source=post_page---top_nav_layout_nav-----------------------------------------)

[Sign in](/m/signin?operation=login&redirect=https%3A%2F%2Fmedium.com%2Fdata-science-in-your-pocket%2Finternvl-3-5-best-open-sourced-multi-modal

{'input': 'Open Source Multimodal LLM Model 추천해줘',
 'output': '다음은 현재 사용 가능한 주요 오픈소스 멀티모달 LLM 모델 추천 목록입니다. 각 모델은 특정 강점과 활용 사례를 가지고 있습니다.\n\n1.  **InternVL 3.5**: 최고의 오픈소스 멀티모달 모델 중 하나로 평가되며, 1B부터 241B에 이르는 다양한 크기로 제공됩니다. MMMU, ChartQA, DocVQA 등 여러 벤치마크에서 뛰어난 성능을 보여줍니다. 영상 이해, 문서 분석, 차트 해석 등 복잡한 시각적 추론 작업에 특히 강력합니다.\n\n2.  **Gemma 3 (Google DeepMind)**: Google에서 개발한 경량 오픈 모델로, 1B, 4B, 12B, 27B 크기가 있습니다. 140개 이상의 언어를 지원하며, 이미지 및 짧은 영상 이해와 텍스트 출력에 특화되어 있습니다. 에이전트 워크플로우를 위한 함수 호출 및 구조화된 출력을 지원합니다.\n\n3.  **GLM-4.5V (Z.ai)**: 106B 파라미터(12B 활성) 모델로, 42개 공개 비전 언어 벤치마크에서 최첨단 성능을 달성했습니다. \'Thinking Mode\'를 통해 깊은 멀티모달 추론을 수행하며, 고급 GUI 에이전트 기능과 시각적 현지화(Grounding)에 강합니다. (현재 영어와 중국어만 지원합니다.)\n\n4.  **Qwen3-VL (Alibaba Cloud)**: Qwen 시리즈의 최신 버전으로, 향상된 멀티모달 추론, 에이전트 기능, 긴 컨텍스트 이해를 제공합니다. 시각 에이전트 기능, 32개 언어를 지원하는 OCR, 256K 토큰의 긴 컨텍스트 윈도우를 통한 고급 영상 이해가 특징입니다.\n\n5.  **Pixtral (Mistral AI)**: Mistral AI의 첫 멀티모달 모델로, 12B 및 124B 파라미터 크기가 있습니다. 이미지와 텍스트 데이터를 효과적으로 처리하며, 뛰어난 지침 준수 능력과 다중 이미지 처리 능력이 강점입니다.\

In [84]:
agent_executor.invoke({'input':"레오나르도 디카프리오의 출생년도를 찾은 뒤, 각 숫자를 순서대로 곱해줘."})



> Entering new AgentExecutor chain...
Thought: 먼저 레오나르도 디카프리오의 출생년도를 찾아야 합니다. 이를 위해 `tavily_search` 도구를 사용하겠습니다.
Action:
```
{
  "action": "tavily_search",
  "action_input": {
    "query": "레오나르도 디카프리오 출생년도"
  }
}
```POSSIBLE ANSWER: 레오나르도 디카프리오는 1974년 11월 11일에 태어났습니다. 그는 미국 출신의 배우입니다. 그의 본명은 레오나르도 빌헬름 디카프리오입니다.

TITLE: 레오나르도 디카프리오 - 나무위키
URL:https://namu.wiki/w/%EB%A0%88%EC%98%A4%EB%82%98%EB%A5%B4%EB%8F%84%20%EB%94%94%EC%B9%B4%ED%94%84%EB%A6%AC%EC%98%A4
Content:본명. Leonardo Wilhelm DiCaprio 레오나르도 빌헬름 디카프리오[1] ; 출생. 1974년 11월 11일 (50세) ; 출생 · 캘리포니아 주 로스앤젤레스 할리우드 ; 국적.
---
TITLE: 레오나르도 디카프리오 - Gaminme - 티스토리
URL:https://gaminmint.tistory.com/entry/%EB%A0%88%EC%98%A4%EB%82%98%EB%A5%B4%EB%8F%84-%EB%94%94%EC%B9%B4%ED%94%84%EB%A6%AC%EC%98%A4
Content:# [Gaminme](https://gaminmint.tistory.com/)

![블로그 이미지](https://tistory1.daumcdn.net/tistory/561722/attach/0d6a1cf23d6749c9af430acb8d4fc533)

# 레오나르도 디카프리오

![](https://blog.kakaocdn.net/dna/bqculS/btqNQbfI4Aw/AAAAAAAAAAAAAAAAAAAAAK0CCdFjlf7IJ

{'input': '레오나르도 디카프리오의 출생년도를 찾은 뒤, 각 숫자를 순서대로 곱해줘.',
 'output': '레오나르도 디카프리오의 출생년도 1974년의 각 숫자를 순서대로 곱한 값은 252입니다.'}

랭체인의 에이전트 구조는 매우 간결하지만, 컨트롤하기가 매우 어려운데요.    
실제로 우리가 개발하는 구조에서는, 모든 단계에서 LLM이 컨텍스트를 저장하여 판단을 내릴 필요는 없습니다.   
또한, 모든 과정이 자연어로 구성되기 때문에 중간에 파싱 오류가 발생하는 경우에 동작이 멈추기도 합니다.   


특정 작업을 반복 실행하거나, 정해진 순서에 따라 실행해야 하는 상황이라면 어떻게 해야 할까요?    
랭체인으로 이와 같은 기능을 구현하는 것도 가능하지만, 다소 복잡한데요.   


랭그래프를 이용하면 구체적인 Workflow를 바탕으로 Agent 형태의 어플리케이션을 효과적으로 만들 수 있습니다.   
